### Yelp Content-Based Recommender Engine

In [1]:
import pandas as pd, numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet');
import help_function

import pickle

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bentleyou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
Tfidf_vectorizer = pd.read_pickle('Tfidf.pkl')

In [3]:
lsa = pd.read_pickle('lsa_model.pkl')
doc_topic_lsa = pd.read_pickle('doc_topic_lsa.pkl')

In [4]:
nmf = pd.read_pickle('nmf_model.pkl')
doc_topic_nmf = pd.read_pickle('doc_topic_nmf.pkl')

#### Text Preprocessing

In [44]:
def preprocessing(texts):
    
    lemmatizer = WordNetLemmatizer()

    return [''.join([(lemmatizer.lemmatize(w)) for w in texts])]

#### Using LSA transformer

In [45]:
def text_transformer_lsa(preprocessed_texts):
    
    vectorized_text = Tfidf_vectorizer.transform(preprocessed_texts)
    
    compressed_vector = lsa.transform(vectorized_text)
    
    return compressed_vector

#### Using NMF transformer

In [46]:
def text_transformer_nmf(preprocessed_texts):
    
    vectorized_text = Tfidf_vectorizer.transform(preprocessed_texts)
    
    compressed_vector = nmf.transform(vectorized_text)
    
    return compressed_vector

In [47]:
def find_similarity_LSA(search, top_search, location):
    
    preprocessed_search = preprocessing(search)
    
    search_vector = text_transformer_lsa(preprocessed_search)
    
    location_res_topic = doc_topic_lsa[doc_topic_lsa['city'] == location].iloc[:,0:10]
    restaurant_topic_array = location_res_topic.values
    restaurant_index = location_res_topic.index
    
    print(restaurant_topic_array.shape)
    cosine_list = []
    
    
    for restaurant in restaurant_topic_array:
        
        cosine_list.append(cosine_similarity([restaurant,search_vector.reshape(-1)])[1][0])
    
    cosine_array = np.array(cosine_list)
    
    restaurant_sim = pd.DataFrame(cosine_array, 
                                  index = restaurant_index,
                                 columns = ['Similarity']).sort_values(by = 'Similarity', 
                                                                       ascending = False)
    return restaurant_sim[:top_search]

In [48]:
def find_similarity_NMF(search, top_search, location):
    
    preprocessed_search = preprocessing(search)

    
    search_vector = text_transformer_nmf(preprocessed_search)
    
    location_res_topic = doc_topic_nmf[doc_topic_nmf['city'] == location].iloc[:,0:10]
    restaurant_topic_array = location_res_topic.values
    restaurant_index = location_res_topic.index
    
    print(restaurant_topic_array.shape)
    cosine_list = []
    
    
    for restaurant in restaurant_topic_array:
        
        cosine_list.append(cosine_similarity([restaurant,search_vector.reshape(-1)])[1][0])
    
    cosine_array = np.array(cosine_list)
    
    restaurant_sim = pd.DataFrame(cosine_array, 
                                  index = restaurant_index,
                                 columns = ['Similarity']).sort_values(by = 'Similarity', 
                                                                       ascending = False)
    return restaurant_sim[:top_search]

In [18]:
find_similarity_NMF('Salad and pizza', 5, 'Las Vegas')

NameError: name 'WordNetLemmatizer' is not defined

In [3]:
find_similarity_LSA('pizza and salad', 5, 'Las Vegas')

NameError: name 'find_similarity_LSA' is not defined

In [2]:
help_function.find_similarity_LSA('pizza and salad', 5, 'Las Vegas')

(6455, 10)


,Similarity
name,
Cugino's Pizzeria,0.975713
Steve-O's New York Pizza,0.971231
Rollin' Pizza Pies,0.969228
Red Brick Pizza,0.968324
Sicilian Pizza II,0.967341
